### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집 방법
- 동이름 입력 -> 매물정보를 데이터프레임으로 수집

#### 절차
 - 동이름 입력 -> 위도, 경도
 - 위도, 경도 -> geohash code(지역범위)
 - geohash -> 매물 아이디 가져오기
 - 매물 아이디 -> 매물 정보 가져오기

In [1]:
import requests
import pandas as pd

### 1. 동이름으로 위도 경도 가져오기

In [5]:
query = "오산시 원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸"
response = requests.get(url)
datas = response.json()['items'][0]
lat, lng = datas['lat'], datas['lng']
lat, lng

(37.141143798828125, 127.07926177978516)

### 2. 위도 경도로 geohash code 가져오기

In [6]:
!pip install geohash2

  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15544 sha256=3fa56be6cbfa43ee23a4e15a05fb7af9d19ace219936317b11af7e7235e5a648
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\2d\0c\66\dcd768c9e7c26fc81ec59bfe24af9d780fc9dbdf7f90b0cf00
Successfully built geohash2


In [8]:
import geohash2

In [11]:
# precision 영역 값이 커질수록 영역이 작아짐
code = geohash2.encode(lat, lng, precision=5)
code

'wyd77'

### 3. geohash로 매물아이디 가져오기

In [13]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [17]:
datas = response.json()["items"]
datas[:3]

[{'lat': 37.13412263086823, 'lng': 127.08478867100307, 'item_id': 30708429},
 {'lat': 37.13368456578751, 'lng': 127.08457983681409, 'item_id': 30217883},
 {'lat': 37.135224407213244, 'lng': 127.08369620861865, 'item_id': 30474918}]

In [19]:
ids = []
for data in datas:
    ids.append(data['item_id'])

ids[:5]

[30708429, 30217883, 30474918, 30681715, 30666077]

In [20]:
ids = [data['item_id'] for data in datas]
ids[:4]

[30708429, 30217883, 30474918, 30681715]

In [23]:
# list comprehension : 간단한 리스트 데이터를 만들때 주로 사용
# 홀수 데이터만 제곱해서 리스트로 만들어 출력
# for문보다 list comprehension이 더 효율적이다

d1 = range(10)
list(d1)

d2 = [ data**2 for data in d1 if data%2!=0]
d2

[1, 9, 25, 49, 81]

### 4. 매물 아이디 -> 매물 정보 가져오기

In [25]:
ids[:2]

[30708429, 30217883]

In [26]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {"domain": "zigbang", 
         "withCoalition": "true",
         "item_ids" : ids,
         }
response = requests.post(url, params)
response

<Response [200]>

In [43]:
items = response.json()["items"]
item_df = pd.DataFrame(items)
item_df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
870,None,30536348,https://ic.zigbang.com/ic/items/30536348/1.jpg,전세,전세,22000,0,72.73,"{'m2': 72.73, 'p': '22'}","{'m2': 66.12, 'p': '20'}",...,False,True,빌라,[추천],경기도 오산시 금암동,None,None,7,2022-02-19T00:54:01+09:00,False
871,None,30720941,https://ic.zigbang.com/ic/items/30720941/1.jpg,월세,월세,300,39,23.14,"{'m2': 23.14, 'p': '7'}","{'m2': 23.14, 'p': '7'}",...,False,True,원룸,[],경기도 오산시 내삼미동,None,None,6,2022-02-20T22:17:51+09:00,True


In [46]:
item_df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new'],
      dtype='object')

In [47]:
columns = ['sales_type', 'deposit', 'rent','address','size_m2', 'manage_cost']

In [48]:
result_df = pd.DataFrame(items)[columns]
result_df

,sales_type,deposit,rent,address,size_m2,manage_cost
0,전세,5500,0,오산시 고현동,23.1400,6
1,월세,300,30,오산시 고현동,19.8347,5
2,월세,500,50,오산시 고현동,26.4400,6
3,월세,300,42,오산시 고현동,26.4500,6
4,월세,300,35,오산시 고현동,19.8300,5
...,...,...,...,...,...,...
867,월세,50,43,오산시 내삼미동,33.0600,7
868,월세,300,35,오산시 내삼미동,19.8300,6
869,전세,22000,0,오산시 금암동,72.7300,7
870,전세,22000,0,오산시 금암동,72.7300,7


In [52]:
#result_df['address'].str.contains(query)
result_df = result_df[result_df['address'].str.contains(query)]
result_df = result_df.reset_index(drop=True)
result_df

,sales_type,deposit,rent,address,size_m2,manage_cost
0,월세,500,35,오산시 원동,23.14,6
1,월세,50,34,오산시 원동,19.83,6
2,월세,300,40,오산시 원동,19.83,6
3,월세,200,29,오산시 원동,26.45,3
4,월세,300,25,오산시 원동,19.83,3.5
...,...,...,...,...,...,...
108,전세,5000,0,오산시 원동,23.14,6
109,전세,5000,0,오산시 원동,23.14,6
110,전세,5000,0,오산시 원동,23.14,6
111,전세,5000,0,오산시 원동,26.45,6


In [55]:
# 보증금 1억 이하, 월세 100만원 이하 매물을 찾고 싶다
result_df[(result_df['deposit'] <= 10000) & (result_df['rent'] >= 37 )]

,sales_type,deposit,rent,address,size_m2,manage_cost
2,월세,300,40,오산시 원동,19.83,6
7,월세,500,45,오산시 원동,46.28,7
8,월세,500,48,오산시 원동,26.45,4.5
9,월세,500,48,오산시 원동,26.45,6
10,월세,300,45,오산시 원동,54.45,4.5
...,...,...,...,...,...,...
101,월세,500,38,오산시 원동,23.14,6
102,월세,500,38,오산시 원동,23.14,6
103,월세,500,38,오산시 원동,23.14,6
104,월세,500,38,오산시 원동,25.00,6


In [70]:
def oneroom(address):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
    response = requests.get(url)
    datas = response.json()['items'][0]
    lat, lng = datas['lat'], datas['lng']
    code = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    datas = response.json()["items"]
    
    ids = [data['item_id'] for data in datas]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
         "domain": "zigbang", 
         "withCoalition": "true",
         "item_ids" : ids[:998],
         }
    response = requests.post(url, params)
    items = response.json()["items"]
    item_df = pd.DataFrame(items)
    
    columns = ['sales_type', 'deposit', 'rent','address'
               ,'size_m2', 'manage_cost']
    
    result_df = item_df[columns]
    result_df = result_df[result_df['address'].str.contains(address)]
    return result_df.reset_index(drop=True)

In [71]:
oneroom("오산시 원동")


,sales_type,deposit,rent,address,size_m2,manage_cost
0,월세,500,35,오산시 원동,23.14,6
1,월세,50,34,오산시 원동,19.83,6
2,월세,300,40,오산시 원동,19.83,6
3,월세,200,29,오산시 원동,26.45,3
4,월세,300,25,오산시 원동,19.83,3.5
...,...,...,...,...,...,...
108,전세,5000,0,오산시 원동,23.14,6
109,전세,5000,0,오산시 원동,23.14,6
110,전세,5000,0,오산시 원동,23.14,6
111,전세,5000,0,오산시 원동,26.45,6
